# Using of Processor

## New Document

In [1]:
from processor import Preprocessor
from FlagEmbedding import BGEM3FlagModel
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import torch

In [2]:
preprocessor = Preprocessor("sample_doc.pdf")

In [3]:
text = preprocessor.read_text()

In [4]:
clean_text = preprocessor.clean_text(text=text)

In [5]:
chunks = preprocessor.split_into_chunks(text=clean_text)

In [6]:
preprocessor.save_chunks(chunks=chunks, path="/home/murad/Documents/self-study/contextual_embeddings/chunks")

In [7]:
xlm_tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
xlm_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")

In [8]:
bge_model = BGEM3FlagModel(model_name_or_path="BAAI/bge-m3",
                           use_fp_16=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

### XLM RoBERTa comparison

In [9]:
xlm_model.eval()
xlm_embeddings = []

with torch.no_grad():
    for chunk in chunks:
        inputs = xlm_tokenizer(chunk, return_tensors="pt", padding=True)
        outputs = xlm_model(**inputs)
        chunk_embedding = outputs.last_hidden_state[:, 0, :]
        xlm_embeddings.append(chunk_embedding)

In [10]:
xlm_collection = preprocessor.create_database(chunks=chunks,
                                              embeddings=xlm_embeddings)

Using ChromaDB directory: /tmp/tmpp2y4j9dy
Successfully stored 27 chunks with embeddings.
Database location: /tmp/tmpp2y4j9dy


In [11]:
query = "Nəzarətsiz öyrənmə nədir?"

In [12]:
xlm_model.eval()

with torch.no_grad():
    inputs = xlm_tokenizer(query, return_tensors="pt", padding=True)
    outputs = xlm_model(**inputs)
    query_embedding = outputs.last_hidden_state[:, 0, :].squeeze(0).numpy().tolist()

In [13]:
xlm_result = xlm_collection.query(
    query_embeddings = query_embedding,
    n_results=5,
    include=["distances", "metadatas", "documents"]
)

In [14]:
xlm_result["documents"]

[['Kompüter Görmə Modelləri və Alqoritmləri Kompüter görmə sahəsində istifadə olunan əsas modellər və alqoritmlər aşağıdakılardır:• Konvolyusional Sinir Şəbəkələri (CNN): Görüntülərin təsnifatı və obyekt tanıma üçün ən geniş istifadə olunan dərin öyrənmə modeli. • YOLO (You Only Look Once): Real vaxt rejimində obyekt tanıma üçün istifadə olunan model. • R-CNN (Region-based CNN): Görüntüdəki obyektlərin yerini təyin etmək və tanımaq üçün istifadə olunan model. • U-Net: Tibbi görüntülərdə seqmentasiya üçün istifadə olunan model. • OpenPose: İnsan bədən quruluşunu təyin etmək üçün istifadə olunan model. • Face Recognition: Üz tanıma üçün istifadə olunan modellər və alqoritmlər. 6.3. Azərbaycanda Kompüter Görmənin Tətbiqi Azərbaycanda kompüter görmə texnologiyaları müxtəlif sahələrdə tətbiq olunur: • Təhlükəsizlik: Videomüşahidə sistemlərində üz tanıma, şübhəli davranışların aşkarlanması. • Nəqliyyat: Avtomobil nömrələrinin tanınması, trafik monitorinqi, sürət pozuntularının aşkarlanması.'

### BGE-M3 Model

In [15]:
bge_embeddings = bge_model.encode(
    sentences=chunks,
    batch_size=12,
    max_length=8192
)["dense_vecs"]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [16]:
bge_collection = preprocessor.create_database(
    chunks=chunks,
    embeddings=bge_embeddings
)

Using ChromaDB directory: /tmp/tmpa1i8k73z
Successfully stored 27 chunks with embeddings.
Database location: /tmp/tmpa1i8k73z


In [17]:
query_embedding_bge = bge_model.encode(
    sentences=query,
    batch_size=12,
    max_length=1024
)["dense_vecs"]
query_embedding_bge = query_embedding_bge.tolist()

In [18]:
bge_result = bge_collection.query(
    query_embeddings=query_embedding_bge,
    n_results=5,
    include=["distances", "metadatas", "documents"]
)

In [19]:
bge_result["documents"]

[['• Dəstək Vektor Maşınları (Support Vector Machines): Məlumatları müxtəlif kateqoriyalara ayıran optimal hiperplanlər tapır. • K-ən Yaxın Qonşu (K-Nearest Neighbors): Yeni nümunələri ən yaxın qonşularının xüsusiyyətlərinə əsasən təsnif edir. 4.2. Nəzarətsiz Öyrənmə (Unsupervised Learning) Nəzarətsiz öyrənmədə, alqoritm yalnız giriş məlumatları əsasında öyrənir və bu məlumatlardakı struktur və əlaqələri aşkar etməyə çalışır. Bu növ öyrənmə, məlumatların klasterləşdirilməsi və ya gizli strukturların aşkarlanması üçün faydalıdır. Əsas nəzarətsiz öyrənmə alqoritmlərinə aşağıdakılar daxildir: • K-Means: Məlumatları K sayda klasterə bölür və hər bir nöqtəni ən yaxın klaster mərkəzinə aid edir. • Hierarchical Clustering: Məlumatları iyerarxik struktura malik klasterlərə bölür. • Principal Component Analysis (PCA): Məlumatların ölçülərini azaldaraq əsas komponentləri saxlayır. • Association Rule Learning: Məlumatlar arasındakı əlaqə qaydalarını aşkar edir, məsələn, birlikdə alınan məhsulları